# AST 384C - Computational Astrophysics - HW 3
## Carlos Jurado

In [89]:
# Python package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time
import scipy.spatial 

# Loading in style file for plots
plt.style.use('/Users/caj3577/Desktop/plotting.mplstyle')

# Problem 1: using KTrees

Set up a code that can generate (uniform) random postions in 3D for a set of N equal-mass
particles in the range [0, 100] in each dimesion. Make sure to use a random number
generator with a seed state that you can specify so that these initial positions are repeatable
exactly.
calculate the acceleration of every particle assuming vacuum boundary conditions (i.e., you
don’t have to worry about periodic boundaries or anything like that) both directly and
using a KDTree for N ∈103,4,5,6. How does the timing scale for each method? Do your
calculations depend on the method you used?

If we let $GM = 1$ in arbitrary units, then the acceleration felt by a particle at position, $r_i$, is given as:

$$ 
\vec{a_i} = \sum_{j=0}^{N} \left( \frac{-\vec{r_{ij}}}{|\vec{r_{ij}}|^3}\right), i \neq j
$$

In [122]:

np.random.seed(1234) #initalize random seed for reproducibility

N_arr = [10**3, 10**4, 10**5]

#i - represents an individual particle 
#j - represents a pair interaction on the ith particle 

for N in N_arr:
    #Create the numpy array structure to hold the position (acceleration) vectors for all particles
    r_vecs, acceleration_vecs = np.random.uniform(0,100,size=(N, 3)), np.zeros((N, 3)) 
    #print(r_vecs.shape)
    # Create the numpy array structure to hold the displacement (magnitudes) for all particles from one another
    displacement_vecs,  distances = np.zeros((N, N, 3)), np.zeros((N, N))    
    #print(displacement_vecs.shape)

    start_time = time.time()
    for i in range(N): #loop through each particle
        for j in range(N): #loop through each interaction acting on a given particle

            #Calculate the displacement between a particle and one of the other particles
            displacement_vecs[i,j] = r_vecs[j] - r_vecs[i] 

            #compute the distance from the displacement vector
            distances[i,j] = np.sqrt(displacement_vecs[i,j,0]**2 + displacement_vecs[i,j,1]**2 + displacement_vecs[i,j,2]**2) #calculate the distance from displacement_vecs

        #ignore the self-interaction term
        mask = distances[i] > 0

        #calculate the sum of the accelerations on one particle due to all of the other particles
        acceleration_vecs[i] = np.sum( displacement_vecs[i][mask] * (1.0 / distances[i][mask]**3)[:, np.newaxis], axis=0 ) 
    end_time = time.time()

    print(f"Directly: The time taken to setup {N} particles is {np.round(end_time - start_time,2)} seconds")



    ###Below is the KD Tree Implementation 
    start_time = time.time() 
    acceleration_vecs_KD = np.zeros((N, 3))
    displacement_vecs_KD, distances_KD = np.zeros((N, N, 3)), np.zeros((N, N))

    my_KD = scipy.spatial.cKDTree(r_vecs) #initalize my KD tree
    for i in range(N):
        #calculate the distance of the nearest neighbor for the N nearests neighbors. In other words, I am getting back 
        #a ascending ordered list of distances for particle pairings on the ith particle
        distances_KD[i], indices = my_KD.query(r_vecs[i], k=N) 

        for j in range(N):
            #Calculate the displacement vectors between a particle and all other particles 
            displacement_vecs_KD[i,j] = r_vecs[j] - r_vecs[i]

        mask = distances_KD[i] > 0 #ignore the self-interaction term

        #calculate the sum of the accelerations on one particle due to all of the other particles
        acceleration_vecs[i] = np.sum( displacement_vecs_KD[i][mask] * (1.0 / distances_KD[i][mask]**3)[:, np.newaxis], axis=0 )
    end_time = time.time()

    print(f"KD Tree: The time taken to setup {N} particles is {np.round(end_time - start_time,2)} seconds")

    print('=====================NEW ITERATION==========================')
    
    
    

'''
###Below is the KD Tree Implementation 
my_KD = scipy.spatial.cKDTree(r_vecs)

for i in range(N):
        distances, indices = my_KD.query(r_vecs[i], k=N)

        mask = indices != i  # Exclude self

        # Compute acceleration using KDTree results
        for j in indices[mask]:  
            displacement = r_vecs[j] - r_vecs[i]  
            distance = np.linalg.norm(displacement)
            if distance > 0:
                acceleration_vecs[i] += displacement * (1.0 / distance**3)  

end_time = time.time()
print(f"KDTree method: Time taken for {N} particles = {np.round(end_time - start_time, 2)} seconds")'
'''

Directly: The time taken to setup 1000 particles is 2.72 seconds
KD Tree: The time taken to setup 1000 particles is 1.45 seconds
=====================NEW ITERATION==========================
Directly: The time taken to setup 10000 particles is 269.75 seconds
KD Tree: The time taken to setup 10000 particles is 127.57 seconds
=====================NEW ITERATION==========================


KeyboardInterrupt: 